In [14]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import psycopg2
from psycopg2 import sql
import json 

In [16]:
""" FUNCIONES """



# 1. Conexión con BD
def conexionBDPostgresSQL():
    # Se omiten los datos de conexión.
    conexion = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="serRIB99",
        host="localhost",
        port="5432"
    )
    return conexion

# 1.2 Insertar datos Emisiones
def Emisiones_Preprocesado_InsertBD_SQLALCHEMY(dataframeEmisiones):
  # Datos de conexión a POSTGRESQL
  usuario = 'postgres'
  password = 'serRIB99'
  servidor = 'localhost'  
  puerto = '5432'
  basedatos = 'postgres'

  # Crear la URL de conexión
  urlConexion = f'postgresql+psycopg2://{usuario}:{password}@{servidor}:{puerto}/{basedatos}'
  # Motor conexión Postgresql
  engine = create_engine(urlConexion)

  # Tabla y Esquema de BD
  esquemaBD = 'tfm'
  tabla = 'emisiones_procesada'

  # Insertar en BBDD desde un Dataframe
  dataframeEmisiones.to_sql(tabla, engine, schema=esquemaBD, if_exists='append', index=False)

# 3. Función para ejecutar cualquier comando SQL
def ejecutarComandoSQL(comandoSQL):
  connection = conexionBDPostgresSQL()
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: {error}")
  finally:
      if connection:
          cursor.close()
          connection.close()

# 2. # Obtener datos Emisiones desde BBDD y generar Dataframe y CSV
def Preprocesamiento_EmisionesEnergia_ToDataframe():
  # Consultar diferentes tipos de energía
  comandoSQL = """
    SELECT fechaEmisiones, tipoenergia, valoremisiones
    FROM tfm.emisiones e
    WHERE e.fechaEmisiones BETWEEN '2011-01-01' AND '2024-12-31'
  """
  registrosEmisiones = ejecutarComandoSQLSelect(comandoSQL)

  # Diccionario dónde insertar los datos
  datosEmisiones = {
      'fecha': [],
      'Carbon': [],
      'CicloCombinado': [],
      'Cogeneracion': [],
      'FuelGas': [],
      'MotorDiesel': [],
      'TurbinaGas': [],
      'TurbinaVapor': [],
      'ResiduosNoRenov': [],
      'EmisionesTotal': [],
      'EmisionesPorMWh' : []
  }

  # Diccionario para Mapear tipo de energía a nombre de columna
  conversionTiposEmisionesEnergia = {
    'Carbón': 'Carbon',
    'Ciclo combinado': 'CicloCombinado',
    'Cogeneracion': 'Cogeneracion',
    'Fuel + Gas': 'FuelGas',
    'Motores diésel': 'MotorDiesel',
    'Nuclear': 'Nuclear',
    'Turbina de gas': 'TurbinaGas',
    'Turbina de vapor': 'TurbinaVapor',
    'Residuos no renovables': 'ResiduosNoRenov',
    'Total tCO2 eq.' : 'EmisionesTotal',
    'tCO2 eq./MWh' :'EmisionesPorMWh' 
  }

  # Procesar los resultados y agregar los datos al diccionario
  for registro in registrosEmisiones:
    fecha = registro[0]
    tipoenergia = registro[1]
    valor_Emisiones = registro[2]
    if fecha not in datosEmisiones['fecha']:
        datosEmisiones['fecha'].append(fecha)
        for key in datosEmisiones.keys():
            if key != 'fecha':
                datosEmisiones[key].append(0) 

    index = datosEmisiones['fecha'].index(fecha)
    columna = conversionTiposEmisionesEnergia.get(tipoenergia, None)

    if columna:
        datosEmisiones[columna][index] = valor_Emisiones

  # Crear dataframe desde "Datos Emisiones"
  dataframeEmisiones = pd.DataFrame(datosEmisiones)

  # DataFrame a CSV
  dataframeEmisiones.to_csv('EmisionesFormateado.csv', index=False)

  # Insertar en BD
  Emisiones_Preprocesado_InsertBD_SQLALCHEMY(dataframeEmisiones)

  
# 4. Función para ejecutar cualquier comando SQL (SELECT)
def ejecutarComandoSQLSelect(comandoSQL):
  connection = conexionBDPostgresSQL()
  resultados = []
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      # Obtener todos los resultados de la consulta
      resultados = cursor.fetchall()

      # Imprimir los resultados
      """for fila in resultados:
          print(fila)"""
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: \n{error}")
  finally:
    if connection:
        cursor.close()
        connection.close()
    return resultados




In [18]:
""" MAIN """

ejecutarComandoSQL("TRUNCATE TABLE tfm.emisiones_procesada;")

Preprocesamiento_EmisionesEnergia_ToDataframe()

Error al ejecutar el comando: no existe la relación «tfm.emisiones_procesada»

Comando ejecutado con éxito: 
    SELECT fechaEmisiones, tipoenergia, valoremis


In [ ]:
# 1. Obtener datos Generación (procesados)
def obtenerDataframeGeneracion():
  comandoSQL_Generacion = "SELECT * FROM tfm.generacion_procesada ORDER BY \"fechaGeneracion\" ASC"
  datosGeneracion, columnasGeneracion =  obtenerDatosBBDD(comandoSQL_Generacion)
  df_Generacion = pd.DataFrame(datosGeneracion, columns=columnasGeneracion)
  return df_Generacion


# 1. Función para generar gráficas de distribución
def generarGraficasDistribucion(dataset):
    numColumnasVisualizacion = 3  # Excluyendo serie temporal
    tiene_serie_temporal = 'fecha' in dataset.columns
    if tiene_serie_temporal:
        numColumnasVisualizacion += 1  # Añadir la columna para serie temporal

    numCaracteristicas = len(dataset.select_dtypes(include=['int', 'float']).columns)
    numFilas = numCaracteristicas

    # Ajustar tamaño de la figura según el número de filas y columnas
    fig, axes = plt.subplots(numFilas, numColumnasVisualizacion,
                            figsize=(numColumnasVisualizacion * 6, numFilas * 5), dpi= 300)
                            # Ajusta el tamaño para la visualización

    # Asegurar que axes siempre sea una matriz bidimensional
    if numFilas == 1:
        axes = [axes]  # Convertir a lista si hay solo una fila

    fila = 0
    for column in dataset.select_dtypes(include=['int', 'float']).columns:
        # Índices de fila y columna para la gráfica actual
        ax = axes[fila]

        # Gráfica de dispersión (Primera columna de la visualización)
        sns.stripplot(x=column, data=dataset, ax=ax[0], s=3)
        ax[0].set_title(f'Dispersión de: {column.upper()}')
        ax[0].set_xlabel(column)

        # Histograma (Segunda columna de la visualización)
        sns.histplot(dataset[column], kde=True, ax=ax[1])
        ax[1].set_title(f'Histograma de: {column.upper()}')
        ax[1].set_xlabel(column)
        ax[1].set_ylabel('Frecuencia')

        # BoxPlot (Tercera columna de la visualización)
        sns.boxplot(data=dataset[column], ax=ax[2])
        ax[2].set_title(f'Boxplot de: {column.upper()}')
        ax[2].set_xlabel(column)
        ax[2].set_ylabel('Valores')

        if tiene_serie_temporal:
            # Serie temporal (Cuarta columna de la visualización, si existe)
            ax[3].plot(dataset['fecha'], dataset[column], linestyle='-')
            ax[3].set_title(f'Serie Temporal de: {column.upper()}')
            ax[3].set_xlabel('Fecha')
            ax[3].set_ylabel('Valor')

        fila += 1

    # Ocultar subplots que no se utilizan
    for fila_restante in range(fila, numFilas):
        for columna in range(numColumnasVisualizacion):
            axes[fila_restante][columna].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
# 1. Generación
dataframeGeneracion = obtenerDataframeGeneracion()
# Asegurar que todos los dataframe tiene un campo llamado "fecha"
dataframeGeneracion = dataframeGeneracion.rename(columns={'fechaGeneracion': 'fecha'})
dataframeGeneracion.dtypes

In [ ]:
# 2. Modificar el tipo de dato para las variables de generación
columnasNumericas = [ 'Carbon_NR', 'CicloCombinado_NR', 'Cogeneracion_NR',
  'Eolica_R', 'FuelGas_NR', 'Hidroeolica_R', 'Hidraulica_R', 'MotorDiesel_NR',
  'Nuclear_NR', 'OtrasRenovables_R', 'SolarFotovoltaica_R', 'SolarTermica_R',
  'TurbinaGas_NR', 'TurbinaVapor_NR', 'TurbinaBombeo_NR', 'ResiduosNoRenov_NR', 'ResiduosRenov_R', 'GeneracionTotal']
#dataframeGeneracion_Copia = dataframeGeneracion.copy()
dataframeGeneracion[columnasNumericas] = dataframeGeneracion[columnasNumericas].apply(pd.to_numeric, errors='coerce')
dataframeGeneracion.dtypes

In [ ]:
dataframeGeneracion.describe(include='all')